## オブザーバビリティを高める機能

必要なライブラリをダウンロードします。

In [ ]:
%pip install -r ../requirements.txt

実行に必要な環境変数を読み込みます。

In [21]:
import os, warnings, time

# おまじない
warnings.filterwarnings("ignore")

endpoint = "http://langfuse-server:3000"
public_key = os.getenv("PUBLIC_KEY")
secret_key = os.getenv("SECRET_KEY")

### プロンプト

各トレースを同一グループにするために、セッションIDをUUID v4ベースで生成する

In [22]:
from langfuse import Langfuse

# Initialize Langfuse client
langfuse = Langfuse(
    public_key=public_key,
    secret_key=secret_key,
    host=endpoint
    )


プロンプトテンプレートの作成

In [23]:
# Create a text prompt
langfuse.create_prompt(
    name="test-sdk",
    type="text",
    prompt="{{topic}}について教えて",
    labels=["sdk","production"],
    config={},
)

In [24]:
langfuse.create_prompt(
    name="test-sdk",
    type="text",
    prompt="簡潔に{{topic}}について教えて",
    labels=["sdk","production"],
    config={},
)

In [25]:

# プロンプトテンプレート`test-sdk"`を取得
langfuse_prompt = langfuse.get_prompt("test-sdk",version=1)

# 取得したプロンプトから
compiled_prompt = langfuse_prompt.compile(topic="大爆笑一発ギャグ")
# -> "As an expert movie critic, do you like Dune 2?"
compiled_prompt

'大爆笑一発ギャグについて教えて'

In [70]:
import uuid

session_id = str(uuid.uuid4())

from langfuse.decorators import langfuse_context, observe

from langchain_core.prompts import ChatPromptTemplate
from langchain_cohere import ChatCohere
from langfuse.callback import CallbackHandler

callback_handler = CallbackHandler(
    public_key=public_key,
    secret_key=secret_key,
    host=endpoint,
    session_id=session_id,
    user_id="user2@example.com"
)
#@observe#(as_type="generation")

@observe()
def nested_generation():
    langchain_prompt = ChatPromptTemplate.from_template(
        langfuse_prompt.get_langchain_prompt(),
        metadata={"langfuse_prompt": langfuse_prompt},
    )
    langfuse_context.update_current_observation(
        prompt=langchain_prompt,
    )
    model = ChatCohere(
        model="command-r-plus",
        temperature=1,
    )
    chain = langchain_prompt | model
    chain.invoke(
        input={"topic": "大爆笑一発ギャグ"},
        config={"callbacks": [callback_handler]},
    )

    #return langchain_prompt

nested_generation()

In [27]:
langfuse.create_dataset(
    name="test",
    # optional description
    description="My first dataset",
    # optional metadata
    metadata={
        "author": "Alice",
        "date": "2022-01-01",
        "type": "benchmark"
    }
)

Dataset(id='cm2ki8l2g001bu5nhzgiebjij', name='test', description='My first dataset', metadata={'date': '2022-01-01', 'type': 'benchmark', 'author': 'Alice'}, project_id='pj-1234567890', created_at=datetime.datetime(2024, 10, 22, 13, 52, 11, 945000, tzinfo=datetime.timezone.utc), updated_at=datetime.datetime(2024, 10, 25, 18, 30, 21, 82000, tzinfo=datetime.timezone.utc))

In [28]:
langfuse.create_dataset_item(
    dataset_name="test",
    # any python object or value, optional
    input={
        "text": "hello world"
    },
    # any python object or value, optional
    expected_output={
        "text": "hello world"
    },
    # metadata, optional
    metadata={
        "model": "llama3",
    }
)

DatasetItem(id='ef21215a-a7c3-4cdf-8b5d-1d1824678a49', status=<DatasetStatus.ACTIVE: 'ACTIVE'>, input={'text': 'hello world'}, expected_output={'text': 'hello world'}, metadata={'model': 'llama3'}, source_trace_id=None, source_observation_id=None, dataset_id='cm2ki8l2g001bu5nhzgiebjij', dataset_name='test', created_at=datetime.datetime(2024, 10, 25, 18, 30, 21, 97000, tzinfo=datetime.timezone.utc), updated_at=datetime.datetime(2024, 10, 25, 18, 30, 21, 97000, tzinfo=datetime.timezone.utc))

In [29]:
langfuse.create_dataset_item(
    dataset_name="test",
    input={ "text": "hello world" },
    expected_output={ "text": "hello world" },
    # link to a trace
    source_trace_id="b35c6305-b639-4670-a755-bfd5e725791f",
    # optional: link to a specific span, event, or generation
    #source_observation_id="<observation_id>"
)

DatasetItem(id='de4bc98c-a6b8-48ec-906c-e8ecae909329', status=<DatasetStatus.ACTIVE: 'ACTIVE'>, input={'text': 'hello world'}, expected_output={'text': 'hello world'}, metadata=None, source_trace_id='b35c6305-b639-4670-a755-bfd5e725791f', source_observation_id=None, dataset_id='cm2ki8l2g001bu5nhzgiebjij', dataset_name='test', created_at=datetime.datetime(2024, 10, 25, 18, 30, 21, 123000, tzinfo=datetime.timezone.utc), updated_at=datetime.datetime(2024, 10, 25, 18, 30, 21, 123000, tzinfo=datetime.timezone.utc))

In [47]:
@observe()
def meet_the_bigbaby():
    print("Meet the BigBaBy!")
    langfuse.score(
        trace_id=langfuse_context.get_current_trace_id(),
        name="Met BB?",
        value=True,
        data_type="BOOLEAN",
    )
meet_the_bigbaby()

Meet the BigBaBy!


In [77]:
from langchain_cohere import ChatCohere
from langchain_core.prompts import ChatPromptTemplate
from langfuse.decorators import langfuse_context

langfuse_context.configure(
    public_key=public_key,
    secret_key=secret_key,
    host=endpoint
)

prompt = ChatPromptTemplate.from_messages(
[
    ("system","簡潔に少ない文字数で返信してください。文字数が少なければ少ないほど良いです。"),
    ("user","{query}"),
]
)
chat = ChatCohere(
    model="command-r-plus",
    temperature=0,
)

chain = prompt | chat
predefined_run_id = str(uuid.uuid4())
res = chain.invoke(
    {"query":"寿限無寿限無?"},
    config={
        "callbacks":[callback_handler],
        "run_id": predefined_run_id,
    }
)

langfuse.score(
    trace_id=predefined_run_id,
    name="content_length",
    value=len(res.content),
    data_type="NUMERIC",
    comment="Length of AIMessage received by user",
)
res.content

'五劫の擦り切れ'

In [75]:
@observe()
def meet_the_bigbaby():
    print("Meet the BigBaBy!")
    langfuse.score(
        trace_id=langfuse_context.get_current_trace_id(),
        name="Met BB?",
        value=True,
        data_type="BOOLEAN",
    )
meet_the_bigbaby()

Meet the BigBaBy!


In [76]:
langfuse.create_dataset(name="japanese_holidays")
local_items = [
    {"input": {"year": "2024","name":"海の日"}, "expected_output": "2024/07/15"},
    {"input": {"year": "2024","name":"天皇誕生日"}, "expected_output": "2024/02/23"},
    {"input": {"year": "2024","name":"勤労感謝の日"}, "expected_output": "2024/11/23"},
]
for item in local_items:
  langfuse.create_dataset_item(
      dataset_name="japanese_holidays",
      input=item["input"],
      expected_output=item["expected_output"]
)

In [72]:
from langchain_cohere import ChatCohere

from langchain_core.prompts import ChatPromptTemplate

def run_japanese_holiday_app(input, callback_handler):
  prompt = ChatPromptTemplate.from_messages(
    [
        ("system","あなたは日本の祝日に詳しい有能なアシスタントです。"),
        ("user","{year}年の{name}をYYYY/MM/DD形式で回答してください。例えば、元日であれば「2024/01/01」と回答してください。"),
    ]
  )
  chat = ChatCohere(
    model="command-r-plus",
    temperature=0,
  )

  chain = prompt | chat

  res = chain.invoke(
    input,
    config={"callbacks":[callback_handler]}
  )

  return res

In [73]:
dataset = langfuse.get_dataset("japanese_holidays")

for item in dataset.items:
  handler = item.get_langchain_handler(run_name="case1")

  completion = run_japanese_holiday_app(item.input, handler)

  exact_match = lambda str1, str2: str1 == str2
  handler.trace.score(
    name="exact_match",
    data_type="BOOLEAN",
    value= exact_match(completion.content, item.expected_output)
  )

TooManyRequestsError: status_code: 429, body: data=None message="You are using a Trial key, which is limited to 10 API calls / minute. You can continue to use the Trial key for free or upgrade to a Production key with higher rate limits at 'https://dashboard.cohere.com/api-keys'. Contact us on 'https://discord.gg/XW44jPfYJu' or email us at support@cohere.com with any questions"